# IMPORT

In [5]:
from PIL import Image
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from datasets import load_dataset
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import torchvision
import torchvision.models as models
from torchsummary import summary
# import cv2  


# AffectNet Dataset Loader

In [2]:
class AffectNetHqDataset(Dataset):
    def __init__(self, dataset, transform=None):
        # 'dataset' is now a subset of the original dataset
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image']
        label = item['label']

        if self.transform:
            image = self.transform(image)

        return image, label

# Load the full dataset
full_dataset = load_dataset("Piro17/affectnethq", split='train')

# Split the dataset into train and test subsets
train_size = int(0.01 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_subset, test_subset = random_split(full_dataset, [train_size, test_size])

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation((-10, 10)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the dataset and dataloader using the subsets
train_dataset = AffectNetHqDataset(Subset(full_dataset, train_subset.indices), transform=train_transform)
test_dataset = AffectNetHqDataset(Subset(full_dataset, test_subset.indices), transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("Dataset Loaded !")

Dataset Loaded !


In [11]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class RAFDBDataset(Dataset):
    def __init__(self, root_dir, label_dir, transform=None):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.transform = transform
        self.labels, self.image_paths = self._load_data()

    def _load_data(self):
        labels = []
        image_paths = []

        labels_file_path = os.path.join(self.label_dir, 'list_partition_label.txt')
        with open(labels_file_path, 'r') as file:
            lines = file.readlines()

            for line in lines:
                parts = line.strip().split(' ')
                label = int(parts[1])
                image_path = os.path.join(self.root_dir, 'aligned', parts[0])
                labels.append(label)
                image_paths.append(image_path)

        return labels, image_paths

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image_path = self.image_paths[idx]
        
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label
    


# Transform function for training data
train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=(-10, 10)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),
])

# Transform function for test data
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Assuming you have already defined full_dataset, train_subset, and test_subset
train_dataset = RAFDBDataset(root_dir="C:/Users/MCE30/Desktop/SAR/M2 SAR/MLA/Projet/RAF-DB/Image/aligned/train", label_dir = r'C:\Users\MCE30\Desktop\SAR\M2 SAR\MLA\Projet\RAF-DB\Image\aligned', transform=train_transform)
test_dataset = RAFDBDataset(root_dir=r'C:\Users\MCE30\Desktop\SAR\M2 SAR\MLA\Projet\RAF-DB\Image\aligned\test', label_dir = r'C:\Users\MCE30\Desktop\SAR\M2 SAR\MLA\Projet\RAF-DB\Image\aligned',transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("RAF-DB Dataset Loaded !")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\MCE30\\Desktop\\SAR\\M2 SAR\\MLA\\Projet\\RAF-DB\\Image\\aligned/list_partition_label.txt'

# PAL Class


In [3]:
class PrivilegedAttributionLoss(nn.Module):
    def __init__(self):
        super(PrivilegedAttributionLoss, self).__init__()

    def forward(self, attribution_maps, prior_maps):
        """
        Compute the Privileged Attribution Loss (PAL).

        Args:
            attribution_maps (torch.Tensor): Attribution maps (a_l) from your model.
            prior_maps (torch.Tensor): Prior maps (a*) that highlight certain regions.

        Returns:
            torch.Tensor: PAL loss value.
        """
        # Calculate mean and standard deviation of attribution maps (a_l)
        mean_al = torch.mean(attribution_maps)
        std_al = torch.std(attribution_maps)

        # Calculate the PAL loss as described in the provided text
        pal_loss = -torch.sum((attribution_maps - mean_al) / std_al * prior_maps)

        return pal_loss

# Modèle VGGFace

In [4]:
class VGGFace(nn.Module):
    def __init__(self):
        super(VGGFace, self).__init__()

        self.conv_layers = nn.Sequential(
            # Premier bloc
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),  
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),  
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Deuxième bloc
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),  
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),  
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Troisième bloc
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),  
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),  
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),  # Ajout de BatchNorm
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Quatrième bloc
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # Ajout de BatchNorm
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # Ajout de BatchNorm
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # Ajout de BatchNorm
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Cinquième bloc
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # Ajout de BatchNorm
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # Ajout de BatchNorm
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # Ajout de BatchNorm
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # self.last_conv_layer = self.conv_layers[-1]
        # self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc_layers = nn.Sequential(
        #     nn.Linear(512, 1024),  
        #     nn.ReLU(inplace=True),
        #     nn.Dropout(0.5),
        #     nn.Linear(1024, 7),
        #     nn.Softmax(dim=1)
        # )

        self.last_conv_layer = self.conv_layers[-1]
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc_layers = nn.Sequential(
            nn.Linear(512, 4096),  
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 7),  # Assuming there are 7 classes in the RAF-DB dataset
            nn.Softmax(dim=1)
        )

        # Attribute to store the gradients of the last convolutional layer
        self.last_conv_gradients = None

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc_layers(x)
        return x

    def capture_last_conv_gradients(self):
        return self.last_conv_layer


# Création des modèles pour les boucles d'entraînements

In [12]:
# Charger le modèle pré-entraîné VGG16
# base_model = torchvision.models.vgg16(pretrained=True)
# Supprimer la dernière couche entièrement connectée
# base_model.classifier = nn.Sequential(*list(base_model.classifier.children())[:-1])
base_model = VGGFace()
# Ajouter une nouvelle couche adaptée à 7 classes
num_classes = 7
classifier_layer = nn.Linear(4096, num_classes)
model = nn.Sequential(base_model, classifier_layer)

# Afficher la structure du modèle
# summary(model, (3, 224, 224))  # Assurez-vous d'ajuster les dimensions en fonction de vos données


# Identifier la dernière couche de convolution
last_conv_layer = base_model.last_conv_layer
# last_conv_layer = model[0].features[28]
print(last_conv_layer)
optimizer = optim.Adam(model.parameters(), lr=4e-5)

# Fonction pour enregistrer le gradient
def save_gradient(grad):
    global conv_output_gradient
    conv_output_gradient = grad

# Attacher un hook pour enregistrer le gradient
last_conv_layer.register_backward_hook(lambda module, grad_in, grad_out: save_gradient(grad_out[0]))
num_epochs = 5

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


# Boucle d'entraînement

In [16]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        optimizer.zero_grad()
        
        # Ensure that images require gradients
        images.requires_grad_()
        
        # Keep labels as integers
        labels = labels.long()
        
        outputs = base_model(images) # Pour VGGFace
        # outputs = model(images)    # Pour VGG16

        # Calculate the gradient of the output with respect to the input
        grad_output = torch.ones(outputs.size(), requires_grad=True)  # Set requires_grad=True
        input_grad = torch.autograd.grad(outputs, images, grad_outputs=grad_output, retain_graph=True)[0]

        # Calculate the gradient times the input
        grad_times_input = input_grad * images
        # Define your attribution maps (a_l) and prior maps (a*) as torch Tensors
        attribution_maps = grad_times_input  
        # prior_maps = torch.tensor(images)   
        prior_maps = images.clone().detach() #Recommandé par une erreur python

        # Create an instance of the PrivilegedAttributionLoss
        pal_loss_fn = PrivilegedAttributionLoss()

        # Calculate the PAL loss
        pal_loss = pal_loss_fn(attribution_maps, prior_maps)
        classification_loss = 0
        # Add the PAL loss to your total loss (cross-entropy or other)
        total_loss = classification_loss + pal_loss

        if epoch == 0 :
            import matplotlib.pyplot as plt
            import numpy as np

            # Assuming grad_times_input is a torch.Tensor and images is a tensor
            grad_times_input_np = grad_times_input[0].cpu().detach().numpy()  # Convert to NumPy array
            grad_times_input_rescaled = (grad_times_input_np - grad_times_input_np.min()) / (grad_times_input_np.max() - grad_times_input_np.min())  # Rescale to [0, 1]

            # Convert the original tensor (images) to a NumPy array for visualization
            original_image_np = images[0].permute(1, 2, 0).cpu().detach().numpy()

            # Create a figure with two subplots
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))

            # Display the original image on the first subplot
            axes[0].imshow(original_image_np)
            axes[0].axis('off')
            axes[0].set_title('Original Image')

            # Display the gradient input as an image on the second subplot
            axes[1].imshow(grad_times_input_rescaled.transpose(1, 2, 0))  # Transpose dimensions if needed
            axes[1].axis('off')
            axes[1].set_title('Gradient Input')

            plt.tight_layout()
            plt.show()




  0%|          | 0/18 [00:03<?, ?it/s]


KeyboardInterrupt: 

# Résultats

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # outputs = model(images) # Pour VGG16
        outputs = base_model(images) # Pour VGGFace
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy on the test set: {100 * correct / total}%')
    
print("résultat")